In [1]:
import numpy as np
import pygl, sys, time
import scipy as sp
from scipy.io import savemat, loadmat
fft2  = np.fft.fft2
randn = np.random.randn
from tqdm import trange
 


In [2]:
Ng, nfac, kh = 32, 1, -.001

In [3]:
class activeModels():
    '''Class to solve a active models'''
    def __init__(self, Nt, dt, dd, rhs):
        self.Nt = Nt
        self.dt = dt
        self.dd = dd
        
        self.rhs = rhs 
        self.XX  = np.zeros((int(self.dd+1), Ng*Ng)) 
        
    def integrate(self, u):
        '''  simulates the equation and plots it at different instants '''
        ii=0
        #fW = open('N%s_z%2.2f_l%2.2f_kh%4.4f_u%2.2f_a%4.4f_fd.txt'%(Ng, zeta, ll, kh, phi0, a), 'w')
        #fW.write('Ng%s_zeta%s_ ll%s_kh%s_phi0%s_a%s_b%s_k%s_dt%s_nfac%s_Ng%s_Nt%s \n'%(Ng, zeta, ll, kh, phi0, a, b,k, dt, nfac, Ng, Nt))
        t1 = time.perf_counter()
        
        for i in trange(self.Nt):          
            if time.perf_counter() - t1 > 42000:
                break    
            u = u + self.dt*self.rhs(u)
            if i%(int(self.Nt/self.dd))==0:  
                self.XX[ii,:] = u.flatten()
                ii += 1 
                #fW.write('%s\n'%u.flatten() )
    

# now set-up the simulation 
a, b, k  = -.25, .25, 1
zeta, ll = 2, -2
phi0 = .6

dim, h, st = 2, 1, 5
Nt, dt, dd = int(1e4), .01, 1000 
Ng2 = Ng*Ng

eta  = 1
grid = {"dim":dim, "Nx":Ng, "Ny":Ng}
ff = pygl.utils.FiniteDifference(grid)
stokes = pygl.solvers.Stokes(eta, grid)

Teff=0.1#nfac = Ng*Ng, np.sqrt(2*Teff/(h*h*dt))
nfac=2


def rhs(u):
    '''
    returns the right hand side of \dot{phi} in active model H
    \dot{u} = Δ(a*u + b*u*u*u + kΔu + λ(∇u)^2) - v.∇u (solve for fluid)
    '''
    #print( t, np.max(u))
    
    u_x=ff.diffx(u);  u_y=ff.diffy(u);  upp=ff.laplacian(u);  
    gp2 = 0.5*(u_x*u_x + u_y*u_y)
    chemPot = -.25*u + .25*u*u*u - upp + 2*ll*gp2
    jx  = -ff.diffx1(chemPot) + nfac*randn(Ng,Ng) + zeta*upp*u_x 
    jy  = -ff.diffy1(chemPot) + nfac*randn(Ng,Ng) + zeta*upp*u_y 
    du  = ff.diffx1(jx) + ff.diffy1(jy)

    sxx = u_x*u_x-gp2;  sxy=u_x*u_y;  syy=u_y*u_y-gp2;
    fx  = ff.diffx(sxx) + ff.diffy(sxy)
    fy  = ff.diffx(sxy) + ff.diffy(syy)

    stokes.solve((fft2(fx), fft2(fy)))
    return -du - kh*(u_x*stokes.vx + u_y*stokes.vy)

In [4]:
am = activeModels(Nt, dt, dd, rhs)
u = phi0 + .0*(1-2*np.random.random((Ng, Ng)))

# run the simulation and save data
t1 = time.perf_counter()
am.integrate(u)
savemat('N%s_z%2.2f_l%2.2f_kh%4.4f_u%2.2f_nf%4.4f_ambp.mat'%(Ng, zeta, ll, kh, phi0, nfac), {'X':am.XX, 'a':a, 'b':b, 'k':k, 'Ng':Ng, 'Nt':am.Nt, 'dt':dt, 'nfac':nfac, 'Tsim':time.perf_counter()-t1})

100%|███████████████████████████████████| 10000/10000 [00:01<00:00, 5565.74it/s]
